# DEMO sprint 5 - Deel 1 (Inspectieresultaten)

### Benodigde Python packages

In [19]:
# import algemeen
from pathlib import Path

import geopandas as gpd

# imports base modules
from toolbox_continu_inzicht.base.config import Config
from toolbox_continu_inzicht.base.data_adapter import DataAdapter
from toolbox_continu_inzicht.inspections import InspectionsToDatabase
from toolbox_continu_inzicht.inspections.filter import Filter
from toolbox_continu_inzicht.inspections.inspections import ClassifyInspections

### Definitie configuratie

**Inlezen configuratie**
<details>
<summary>Configuratie bekijken</summary>   

<pre style="font-size: 10pt">
GlobalVariables:
    rootdir: "data"
    moments: [-24,0,24,48]

    ClassifyInspections:
        classify_column: "conditiescore"

    Filter:
        query: "Status_waarneming=='Afgerond'"
        drop_columns:
            - "Drop"

    InspectionsToDatabase:
        index: 1

DataAdapter:
    default_options:
        csv:
            sep: ";"

    inspectie_points_csv:
        type: csv
        path: "continu-inzicht-inspectie-data.csv"

    inspectie_points_styling_csv:
        type: csv
        path: "continu-inzicht-inspectie-styling.csv"

    inspectie_points_layer_csv:
        type: csv
        path: "continu-inzicht-inspectie-layer.csv"

    inspectie_points_filtered_csv:
        type: csv
        path: "hidden_inspectie_filter.csv"
        index: False

    inspectie_points_filtered_geojson:
        type: shape
        path: "hidden-inspectie-filtered.geojson"
        index: False

    inspectie_points_filtered_styled_geojson:
        type: shape
        path: "hidden-inspectie-filtered-styled.geojson"

    inspectie_points_styled_legenda_csv:
        type: csv
        path: "hidden-inspectie-styled-legenda.csv"
        index: False

    inspectie_points_filtered_styled_ci_to_database:
        type: postgis
        schema: continuinzicht_demo_realtime
        database: "continuinzicht"
        table: demo_inspectie_resultaten
        if_exists: replace

    inspectie_layers_ci_to_database:
        type: postgresql_database
        schema: continuinzicht_demo_realtime
        database: "continuinzicht"
        table: layers
        if_exists: append

</pre>
</details>

### Lees configuratie in en maak dictionary base.Config aan

In [20]:
# lees configuratiebestand
yaml_config_file = "continu-inzicht-demo-inspectie.yaml"
data_path = Path.joinpath(Path.cwd(), "data", yaml_config_file)

config = Config(config_path=data_path)
config.lees_config()

### Geef config mee aan base.Adapter

In [21]:
# data adapter aanmaken
data_adapter = DataAdapter(config=config)

### Lees CSV met inspectiedata met behulp van data-adapter en filter deze

In [22]:
# lees csv met inspecties en filter inspecties
filter = Filter(data_adapter=data_adapter)
filter.run(input="inspectie_points_csv", output="inspectie_points_filtered_csv")
filter.df_out

,ID,Inspectie_Datum,X,Y,Omschrijving,ConditieScore,Image_URL,Status_waarneming
0,1,1012025,92348.0,410723.00,Graverij in de teen van de dijk,gering,https://media.iasset.nl/files/ws-waterveilighe...,Afgerond
1,2,3042025,92819.0,410501.00,Veel onkruid,matig,https://media.iasset.nl/files/ws-waterveilighe...,Afgerond
2,3,23022025,204760.5,429306.74,Scheuren in asfaltbeton,ernstig,https://media.iasset.nl/files/ws-waterveilighe...,Niet afgerond


### Sla gefilterde inspectiedata op als geojson

In [23]:
# sla inspecties op als geojson
# stel geometry samen
geometry = gpd.points_from_xy(filter.df_out["X"], filter.df_out["Y"])

# zet de juiste projectie (EPSG:4326 wordt verwacht)
gdf_in = gpd.GeoDataFrame(filter.df_out, geometry=geometry)
gdf_in = gdf_in.set_crs(epsg=28992)
gdf_in = gdf_in.to_crs(epsg=4326)

# alle attribuut namen in kleine letters zonder spaties
gdf_in.columns = gdf_in.columns.str.lower().str.replace(" ", "_")

# Eventuele URLs naar foto's in attribuut 'Image_URL' opslaan
# gdf_in = gdf_in.rename(columns={"foto_url": "Image_URL"})

output_file = Path(config.global_variables["rootdir"]) / Path(
    config.data_adapters["inspectie_points_filtered_geojson"]["path"]
)

gdf_in.to_file(output_file, driver="GeoJSON")
print(output_file)

data\hidden-inspectie-filtered.geojson


### Pas styling toe op data in geosjon

In [24]:
# lees gefilterde inspectie data in en classificeer a.d.v. styling csv
classify_inspections = ClassifyInspections(data_adapter=data_adapter)
classify_inspections.run(
    input=["inspectie_points_filtered_geojson", "inspectie_points_styling_csv"],
    output=[
        "inspectie_points_filtered_styled_geojson",
        "inspectie_points_styled_legenda_csv",
    ],
)

### Schrijf inspectie data (met styling) naar CI database met data-adapter

In [25]:
# schrijf inspectie data weg naar de CI database (via een data-adapter) -> aparte tabel
gdf_in = data_adapter.input("inspectie_points_filtered_styled_geojson")
data_adapter.output("inspectie_points_filtered_styled_ci_to_database", gdf_in)

### Schrijf kaartlagen configuratie naar CI database met data-adapter

In [26]:
# schrijf kaartlagen menu en styling naar de CI database (via een data-adapter)
inspections_to_database = InspectionsToDatabase(data_adapter=data_adapter)
inspections_to_database.run(
    input=[
        "inspectie_points_filtered_styled_geojson",
        "inspectie_points_styled_legenda_csv",
        "inspectie_points_layer_csv",
    ],
    output="inspectie_layers_ci_to_database",
)